In [1]:
##Verify SDK

import azureml.core
print(f"SDK version: {azureml.core.VERSION}")


## Set up diagnostics
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)


SDK version: 1.15.0
Turning diagnostics collection on. 


In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity-project
Azure region: eastus
Subscription id: 086a16fa-80c1-42e4-aa0e-8627ef0b286e
Resource group: udacity-project


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacity-cluster"

try:
	compute_target = ComputeTarget(workspace=ws, name=cluster_name)
	print('Found existing cluster, use it.')
except ComputeTargetException:
	compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                       	vm_priority = 'lowpriority',
                                                       	max_nodes=4)
	compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
	compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 15)


print(compute_target.get_status().serialize())

Creating
Succeeded..................................................................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-10-13T10:26:19.834000+00:00', 'errors': None, 'creationTime': '2020-10-13T10:26:14.955603+00:00', 'modifiedTime': '2020-10-13T10:26:31.947199+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_D2_V2'}


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import ScriptRunConfig
import os


# Specify parameter sampler
params = {
	"--C": uniform(0.5, 1.0),
	"--max_iter": choice(40, 100, 150)
}

param_sampling =  RandomParameterSampling(params)
### YOUR CODE HERE ###

# Specify a Policy
bandit_policy =    BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    print("Creating new directory")
    os.mkdir("./training")
else:
    print("Directory already existed")

# Create a SKLearn estimator for use with train.py
estimator = SKLearn(
	    source_directory=".",
		compute_target=compute_target,
	    entry_script="train.py"
)   ### YOUR CODE HERE ###


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
	estimator=estimator,
	hyperparameter_sampling=param_sampling,
	policy=bandit_policy,
	primary_metric_name="Accuracy",
	primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
	max_concurrent_runs=4,
	max_total_runs=12)      ### YOUR CODE HERE ###

Directory already existed


In [5]:
!ls ./training/

In [6]:
!cat ./train.py

from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
import logging

# TODO: Create TabularDataset using TabularDatasetFactory
# Data is located at:
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(url)  ### YOUR CODE HERE ###

#x, y = clean_data(ds)

# TODO: Split data into train and test sets.

### YOUR CODE HERE ###
#x_train, x_test, y_train, y_test = train_test_split(x, y)

run = Run.get_context()

def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [8]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO',…

In [9]:
#Ensure the hyperdrive run is complete
hyperdrive_run.wait_for_completion(show_output= True)
assert(hyperdrive_run.get_status() == 'Completed')

RunId: HD_e89b8605-deb9-4d53-a5b5-40e38135b8fc
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_e89b8605-deb9-4d53-a5b5-40e38135b8fc?wsid=/subscriptions/086a16fa-80c1-42e4-aa0e-8627ef0b286e/resourcegroups/udacity-project/workspaces/udacity-project

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-10-13T10:41:27.914192][API][INFO]Experiment created<END>\n""<START>[2020-10-13T10:41:28.798693][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-10-13T10:41:29.276060][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_e89b8605-deb9-4d53-a5b5-40e38135b8fc
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_e89b8605-deb9-4d53-a5b5-40e38135b8fc?wsid=/subscriptions/086a16fa-80c1-42e4-aa0e-8627ef0b286e/resourcegroups/udacity-project/workspaces/udacity-project



In [10]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
best_hyperdrive_run.download_files("./training/best_model_hyper.joblib", "logreg_best_model.joblib")

## Joblib gives thread lock error  so using download_files instead
#joblib.dump(best_hyperdrive_run, 'best_hyper_model.joblib')
#joblib.dump(value=best_hyperdrive_run, filename=os.path.join('./training', 'best_hyperdrive_model.joblib'))




In [11]:
best_run_metrics = best_hyperdrive_run.get_metrics()
parameter_values = best_hyperdrive_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_hyperdrive_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:', parameter_values[1])
print('\n max_iter:', parameter_values[3])


Best Run Id:  HD_e89b8605-deb9-4d53-a5b5-40e38135b8fc_0

 Accuracy: 0.91442097596504

 C: 0.5743940087327192

 max_iter: 150


# Azure AutoMl

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(url)


In [13]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###)
#x['label'] = y


x_train, x_test, y_train, y_test = train_test_split(x, y)
train_data = pd.concat([x_train,y_train],axis = 1)

In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= train_data,
    label_column_name= 'y',
    n_cross_validations= 5)

In [15]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)
RunDetails(hyperdrive_run).show()


Running on local machine
Parent Run ID: AutoML_3c2fbbf0-45af-48c3-8c71-64dfc8b03fe9

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO',…

In [16]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_run, best_automl_model = automl_run.get_output()
print(best_automl_model)

import joblib
joblib.dump(best_automl_model, 'best_automl_model.joblib')

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  l1_ratio=0.3877551020408163,
                                                                                                  learning_rate='constant',
                                                                                                  loss='squared_hinge',
                     

['best_automl_model.joblib']

In [17]:
## Check performance on test dataset
from sklearn.metrics import accuracy_score
y_pred = best_automl_model.predict(x_test)
print(f'Accuracy on test data set is: f{accuracy_score(y_test, y_pred)}')


Accuracy on test data set is: f0.9191551347414421


In [18]:
# Delete Compute cluster
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

